In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch

from torch.autograd import Variable

import torch.utils.data as dataset
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import torch.tensor as tensor
from tqdm import tqdm
import seaborn as sns

import model_2 as model_file
import module_split as module
import RF_module as RF
import scipy.io as sio

#For model class:
import torch.nn.functional as F


In [2]:
# class TotalVariationLoss(object):
#     def __init__(self, device):
        
#         if device >= 0:
#             self.cuda0 = torch.device(f'cuda:{device}')
#         else:
#             self.cuda0 = 'cpu'

#     def tv_loss(self, x):
#         h1 = torch.sum((x[:,:,:,1:]-x[:,:,:,:-1])**2)
#         h2 = torch.sum((x[:,:,1:,:]-x[:,:,:-1,:])**2)

#         y = h1 + h2
        
#         if self.cuda0 != 'cpu':
#             y = y.to(self.cuda0)
        
#         return y 

# class LossFunction(object):
#     def __init__(self, device):
#         if device >= 0:
#             cuda0 = torch.device(f'cuda:{device}')
#             vgg = models.vgg16(pretrained=True).to(cuda0)
#             self.totalVariationLoss = TotalVariationLoss(device)
#         else:
#             vgg = models.vgg16(pretrained=True)
#             self.totalVariationLoss = TotalVariationLoss(device)
        
#         self.mean_sq0 = nn.MSELoss()
#         self.mean_sq1 = nn.MSELoss()

#         self.vgg16Layers = nn.Sequential(*list(vgg.children())[0])[:11]
        
#         for param in self.vgg16Layers.parameters():
#             param.requires_grad = False

#     def __call__(self, t, y, pix = 1, tv=1e-8):
        
#         t_ = self.vgg16Layers(t)
#         y_ = self.vgg16Layers(y)
#         feature = self.mean_sq0(t_, y_)
#         pixel_loss = pix * self.mean_sq1(t, y)

#         total_variation_loss = tv * self.totalVariationLoss.tv_loss(y)
#         loss = feature + pixel_loss + total_variation_loss

#         return loss
    

In [7]:
class Vgg19(torch.nn.Module):
    def __init__(self, requires_grad=False):
        super(Vgg19, self).__init__()
        vgg_pretrained_features = models.vgg19(pretrained=True).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        self.slice5 = torch.nn.Sequential()
        for x in range(2):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(2, 7):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(7, 12):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(12, 21):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        for x in range(21, 30):
            self.slice5.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                
                param = Variable(param, volatile = True)
                
                param.requires_grad = False

    def forward(self, X):
#         X = Variable(X)
        h_relu1 = self.slice1(X)
        h_relu2 = self.slice2(h_relu1)        
        h_relu3 = self.slice3(h_relu2)        
        h_relu4 = self.slice4(h_relu3)        
        h_relu5 = self.slice5(h_relu4)                
        out = [h_relu1, h_relu2, h_relu3, h_relu4, h_relu5]
        return out

class VGGLoss(nn.Module):
    def __init__(self, gpu_ids):
        super(VGGLoss, self).__init__()        
        self.vgg = Vgg19().cuda(device = gpu_ids)
        self.criterion = nn.L1Loss()
        self.weights = [1.0/32, 1.0/16, 1.0/8, 1.0/4, 1.0]        

    def forward(self, x, y):              
        x_vgg, y_vgg = self.vgg(x), self.vgg(y)
        loss = 0
        for i in range(len(x_vgg)):
            loss += self.weights[i] * self.criterion(x_vgg[i], y_vgg[i].detach())        
        return loss

In [3]:
class ResblocksDeconv(nn.Module):
    def __init__(self, in_channels, outsize):
        super(ResblocksDeconv, self).__init__()
#         conv 1_1:
        self.conv1_1 = nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1)
        self.bn1_1 = nn.BatchNorm2d(64) # Maybe eps = 0.001
#         conv 1_2
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn1_2 = nn.BatchNorm2d(64)
#         pool 1
        self.maxp1 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
#         120 x 120
#         conv2_1
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2_1 = nn.BatchNorm2d(128)
#         conv2_2
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn2_2 = nn.BatchNorm2d(128)
#         pool2
        self.maxp2 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)

#         60 x 60
#         conv3_1
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn3_1 = nn.BatchNorm2d(256)
#         conv3_2
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn3_2 = nn.BatchNorm2d(256)
#         conv3_3
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn3_3 = nn.BatchNorm2d(256)
#         pool3
        self.maxp3 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)   
    
    
#         30 x 30
#         conv4_1
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn4_1 = nn.BatchNorm2d(512)
#         conv4_2
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn4_2 = nn.BatchNorm2d(512)
#         conv4_3
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn4_3 = nn.BatchNorm2d(512)
#         pool4
        self.maxp4 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)   
    
#         15 x 15 
#         conv5_1
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn5_1 = nn.BatchNorm2d(512)
#         conv5_2
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn5_2 = nn.BatchNorm2d(512)
#         conv5_3
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn5_3 = nn.BatchNorm2d(512)
#         pool5
        self.maxp5 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)           
     
#         7 x 7 
        self.residualBlock_1 = ResidualBlock(512, 512)
        self.residualBlock_2 = ResidualBlock(512, 512)
        self.residualBlock_3 = ResidualBlock(512, 512)
        self.residualBlock_4 = ResidualBlock(512, 512)
        self.residualBlock_5 = ResidualBlock(512, 512)
        
        
#         7 x 7 
#         unpool5
        self.unpool5 = nn.MaxUnpool2d(2,2) # 
#         14 x 14 (we want 15)
#         deconv5_1
        self.deconv5_1 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.debn5_1 = nn.BatchNorm2d(512)
#         conv3_2
        self.deconv5_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.debn5_2 = nn.BatchNorm2d(512)
#         conv3_3
        self.deconv5_3 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.debn5_3 = nn.BatchNorm2d(512)
        
        
#         _ x _ 
#         unpool4
        self.unpool4 = nn.MaxUnpool2d(2,2) #paper did unpoolsize 28 (we want 30 x 30)
#         _ x _
#         deconv4_1
        self.deconv4_1 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.debn4_1 = nn.BatchNorm2d(512)
#         deconv4_2
        self.deconv4_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.debn4_2 = nn.BatchNorm2d(512)
#         conv4_3
        self.deconv4_3 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1)
        self.debn4_3 = nn.BatchNorm2d(256)        
        
#         _ x _ 
#         unpool3
        self.unpool3 = nn.MaxUnpool2d(2,2) #paper did unpoolsize 56 (we want 60 x 60)
#         _ x _
#         deconv3_1
        self.deconv3_1 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.debn3_1 = nn.BatchNorm2d(256)
#         deconv3_2
        self.deconv3_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.debn3_2 = nn.BatchNorm2d(256)
#         deconv3_3
        self.deconv3_3 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1)
        self.debn3_3 = nn.BatchNorm2d(128)
        
        
#         _ x _ 
#         unpool2
        self.unpool2 = nn.MaxUnpool2d(2,2) #paper did unpoolsize 112 (we want 120 x 120)
#         _ x _
#         deconv3_1
        self.deconv2_1 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.debn2_1 = nn.BatchNorm2d(128)
#         deconv3_2
        self.deconv2_2 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.debn2_2 = nn.BatchNorm2d(64)
        
        
#         _ x _ 
#         unpool1
        self.unpool1 = nn.MaxUnpool2d(2,2) #paper did unpoolsize 224 (we want 240 x 240)
#         _ x _
#         deconv3_1
        self.deconv1_1 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)
        self.debn1_1 = nn.BatchNorm2d(32)
#         deconv3_2
        self.deconv1_2 = nn.Conv2d(32, 3, kernel_size=3, stride=1, padding=1)
        self.debn1_2 = nn.BatchNorm2d(3)
        
        
        
        
        self.in_channels = in_channels
        self.outsize = outsize

    def __call__(self, x):
        print('x:', x.shape)
        h = self.conv1_1(x)
        h = self.bn1_1(h)
        h = F.relu(h)
        h = self.conv1_2(h)
        h = self.bn1_2(h)
        h = F.relu(h)
        h, i1 = self.maxp1(h)
        size1 = h.size()
        print('conv1_2:', h.shape)
        
        h = self.conv2_1(h)
        h = self.bn2_1(h)
        h = F.relu(h)
        h = self.conv2_2(h)
        h = self.bn2_2(h)
        h = F.relu(h)
        h, i2 = self.maxp2(h)
        size2 = h.size()
        print('conv2_2:', h.shape)
        
        h = self.conv3_1(h)
        h = self.bn3_1(h)
        h = F.relu(h)
        h = self.conv3_2(h)
        h = self.bn3_2(h)
        h = F.relu(h)
        h = self.conv3_3(h)
        h = self.bn3_3(h)
        h = F.relu(h)
        h, i3 = self.maxp3(h)
        size3 = h.size()
        print('conv3:', h.shape)
        
        h = self.conv4_1(h)
        h = self.bn4_1(h)
        h = F.relu(h)
        h = self.conv4_2(h)
        h = self.bn4_2(h)
        h = F.relu(h)
        h = self.conv4_3(h)
        h = self.bn4_3(h)
        h = F.relu(h)
        h, i4 = self.maxp4(h)
        size4 = h.size()
        print('conv4:', h.shape)
        
        
        h = self.conv5_1(h)
        h = self.bn5_1(h)
        h = F.relu(h)
        h = self.conv5_2(h)
        h = self.bn5_2(h)
        h = F.relu(h)
        h = self.conv5_3(h)
        h = self.bn5_3(h)
        h = F.relu(h)
#         h, i5 = self.maxp5(h)
        h, i5 = self.maxp5(h)
        size5 = h.size()
        print('conv5:', h.shape)
        

        h = self.residualBlock_1(h)
        h = self.residualBlock_2(h)
        h = self.residualBlock_3(h)
        h = self.residualBlock_4(h)
        h = self.residualBlock_5(h)


        print('resblock:', h.shape)
        h = self.unpool5(h, i5, output_size = size4)
        print('unpool5_want15', h.shape)
        h = self.deconv5_1(h)
        h = self.debn5_1(h)
        h = F.relu(h)
        h = self.deconv5_2(h)
        h = self.debn5_2(h)
        h = F.relu(h)
        h = self.deconv5_3(h)
        h = self.debn5_3(h)
        h = F.relu(h)
        print('deconv5:', h.shape)
        
        h = self.unpool4(h, i4)
        print('unpool4', h.shape)
        h = self.deconv4_1(h)
        h = self.debn4_1(h)
        h = F.relu(h)
        h = self.deconv4_2(h)
        h = self.debn4_2(h)
        h = F.relu(h)
        h = self.deconv4_3(h)
        h = self.debn4_3(h)
        h = F.relu(h)
        print('deconv4:', h.shape)

        h = self.unpool3(h, i3)
        print('unpool3', h.shape)
        h = self.deconv3_1(h)
        h = self.debn3_1(h)
        h = F.relu(h)
        h = self.deconv3_2(h)
        h = self.debn3_2(h)
        h = F.relu(h)
        h = self.deconv3_3(h)
        h = self.debn3_3(h)
        h = F.relu(h)
        print('deconv3:', h.shape)

        
        h = self.unpool2(h, i2)
        print('unpool2', h.shape)
        h = self.deconv2_1(h)
        h = self.debn2_1(h)
        h = F.relu(h)
        h = self.deconv2_2(h)
        h = self.debn2_2(h)
        h = F.relu(h)
        print('deconv2:', h.shape)
        
        
        h = self.unpool1(h, i1)
        print('unpool1', h.shape)
        h = self.deconv1_1(h)
        h = self.debn1_1(h)
        h = F.relu(h)
        h = self.deconv1_2(h)
        h = self.debn1_2(h)
        h = F.relu(h)
        print('deconv1:', h.shape)
        

        y = (torch.tanh(h) + 1) / 2
        print('final_shape:', y.shape)

        return y
    
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.bn1= nn.BatchNorm2d(out_channels)
        self.bn2= nn.BatchNorm2d(out_channels)
        self.conv1 =nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1, bias = False)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1, bias = False)

        self.in_channels = in_channels
        self.out_channels = out_channels

    def __call__(self, x, finetune = False):
        h = self.conv1(x)
        h = self.bn1(h)
        h = F.relu(h)
        
        h = self.conv2(h)
        h = self.bn2(h)
        y = F.relu(h + x)

        return y




In [4]:
# class ResblocksDeconv(nn.Module):
#     def __init__(self, in_channels, outsize):
#         super(ResblocksDeconv, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=5, stride=1, padding=1)
#         self.maxp1 = nn.MaxPool2d(2)
#         self.bn1 = nn.BatchNorm2d(32)

#         self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=1)
#         self.maxp2 = nn.MaxPool2d(2)
#         self.bn2 = nn.BatchNorm2d(64)

#         self.conv3 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=1)
#         self.maxp3 = nn.MaxPool2d(2)
#         self.bn3 = nn.BatchNorm2d(128)

#         self.residualBlock_1 = ResidualBlock(128, 128)
#         self.residualBlock_2 = ResidualBlock(128, 128)
#         self.residualBlock_3 = ResidualBlock(128, 128)
#         self.residualBlock_4 = ResidualBlock(128, 128)
#         self.residualBlock_5 = ResidualBlock(128, 128)
        
#         self.deconv4 = nn.ConvTranspose2d(128, 128, kernel_size=2, stride=1, padding=0)
# #         self.maxp4 = nn.MaxUnPool2d(2)
#         self.bn4 = nn.BatchNorm2d(128)
        
#         self.deconv5 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2, padding=0)
# #         self.maxp5 = nn.MaxPool2d(2)
#         self.bn5 = nn.BatchNorm2d(64)
        
#         self.deconv6 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2, padding=0, bias = False)
# #         self.maxp6 = nn.MaxPool2d(2)
#         self.bn6 = nn.BatchNorm2d(32)
        
#         self.deconv7 = nn.ConvTranspose2d(32, 32, kernel_size=5, stride=2, padding=0, bias=False)
# #         self.maxp7 = nn.MaxPool2d(2)
#         self.bn7 = nn.BatchNorm2d(32)
    
#         self.deconv8 = nn.ConvTranspose2d(32, 3, kernel_size=6, stride = 1, padding=0, bias=False)
# #         self.maxp7 = nn.MaxUnpool2d(9)
#         self.bn8 = nn.BatchNorm2d(3)
    
#         self.in_channels = in_channels
#         self.outsize = outsize

#     def __call__(self, x):
#         print('x:', x.shape)
        
#         h = self.conv1(x)
#         print('conv1_nomx:', h.shape)

#         h = self.maxp1(h)
#         h = self.bn1(h)
#         h = F.relu(h)
#         print('conv1:', h.shape)
        
#         h = self.conv2(h)
#         h = self.maxp2(h)
#         h = self.bn2(h)
#         h = F.relu(h)
#         print('conv2:', h.shape)

        
#         h = self.conv3(h)
#         h = self.maxp3(h)
#         h = self.bn3(h)
#         h = F.relu(h)
#         print('conv3:', h.shape)

#         h = self.residualBlock_1(h)
#         h = self.residualBlock_2(h)
#         h = self.residualBlock_3(h)
#         h = self.residualBlock_4(h)
#         h = self.residualBlock_5(h)
        
# #         h = self.convout(h)
#         h = self.deconv4(h)
# #         h = self.maxp4(h)
#         h = self.bn4(h)
#         h = F.relu(h)
        
#         print('deconv4:', h.shape)
        
#         h = self.deconv5(h)
# #         h = self.maxp5(h)
#         h = self.bn5(h)
#         h = F.relu(h)
#         print('deconv5:', h.shape)


#         h = self.deconv6(h)
# #         h = self.maxp6(h)
#         h = self.bn6(h)
#         h = F.relu(h)
#         print('deconv6:', h.shape)

        
#         h = self.deconv7(h)
# #         h = self.maxp7(h)
#         h = self.bn7(h)
#         h = F.relu(h)
#         print('deconv7:', h.shape)
        
#         h = self.deconv8(h)
# #         h = self.maxp8(h)
#         h = self.bn8(h)
#         y = (torch.tanh(h) + 1) / 2
#         print('deconv8_final:', y.shape)

#         return y
    
# class ResidualBlock(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(ResidualBlock, self).__init__()
#         self.bn1= nn.BatchNorm2d(out_channels)
#         self.bn2= nn.BatchNorm2d(out_channels)
#         self.conv1 =nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1, bias = False)
#         self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1, bias = False)

#         self.in_channels = in_channels
#         self.out_channels = out_channels

#     def __call__(self, x, finetune = False):
#         h = self.conv1(x)
#         h = self.bn1(h)
#         h = F.relu(h)
        
#         h = self.conv2(h)
#         h = self.bn2(h)
#         y = F.relu(h + x)

#         return y




In [5]:
# vgg = models.vgg16(pretrained=True)
# vgglayer = nn.Sequential(*list(vgg.children())[0])[:11]

# for param in vgglayer.parameters():
    
#     param.requires_grad = False
#     print(param.requires_grad)


# param.requires_grad

In [8]:
# -----
# Some params
# -----
device = 1
cuda0 = torch.device(f'cuda:{device}')

batch_size = 4

# -----
# RF gaus maps
# ------
gaus = module.load_gausdata()

seen_images = module.load_ydata()
# ------
# Training
# ------
nn_training = module.load_LFPdata('training')
training_iterator = module.make_iterator(nn_training, 'training', batch_size, shuffle = True)
# ------
# Testing
# ------
nn_testing = module.load_LFPdata('testing')
testing_iterator = module.make_iterator(nn_testing, 'testing', batch_size, shuffle = False)

# -----
# Model, loss, & optimizer
# -----

in_channels=2
model = ResblocksDeconv(in_channels, (240,240))
# model = ran.model
if device >= 0:
    model.cuda(device)
# lossFunction = module.LossFunction(device)
lossFunction = VGGLoss(device)
optimizer = optim.Adam(model.parameters())


if in_channels == 2:
    inputtype = 'V1_V4'
if in_channels == 191:
    inputtype = 'all_channels'
    

/home/lynnle/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


In [ ]:
# y = model(inputs)

# y.shape

# plt.figure(figsize=(10,10))
# plt.subplot(1,2,1)
# plt.imshow(target_batch[0].cpu().detach().numpy().transpose(2,1,0))
# plt.subplot(1,2,2)
# plt.imshow(y[0].cpu().detach().numpy().transpose(2,1,0))

# # padding = 5

In [17]:
### TO TEST THE OUTSHAPE


hori_means, verti_means, std_avg = RF.extract_means_std()

losses_train = []
losses_test = []
epochs = 2
confidence_mask = RF.make_confidence_mask(hori_means, verti_means, std_avg)
confidence_mask = torch.from_numpy(confidence_mask.astype('float32')).to(cuda0)

loss_train = 0

for sample, target_indices in tqdm(training_iterator, total=len(training_iterator)):
    # -----
    # Inputs
    # -----
    mean_signals  = sample[:,300]
    gaus_expand_to_batch = gaus.expand([len(target_indices), 191, 240, 240])
    mean_signals_expand_to_gaus = mean_signals.expand([240, 240, len(target_indices), 191])
    mean_signals_expand_to_gaus = mean_signals_expand_to_gaus.transpose(0,2).transpose(1,3)
    inputs = module.select_type_inputs(inputtype, gaus_expand_to_batch, mean_signals_expand_to_gaus)
    inputs = inputs.to(cuda0)

    # -----
    # Targets
    # -----
    target_batch = seen_images[target_indices]
    target_batch = target_batch.transpose(3,1).transpose(2,3)
    target_batch = target_batch.to(cuda0)

    # -----
    # Outputs
    # -----
    y = model(inputs)


NameError: name 'cuda0' is not defined

In [ ]:

hori_means, verti_means, std_avg = RF.extract_means_std()

losses_train = []
losses_test = []
epochs = 2
confidence_mask = RF.make_confidence_mask(hori_means, verti_means, std_avg)
confidence_mask = torch.from_numpy(confidence_mask.astype('float32')).to(cuda0)
for e in range(epochs):  # loop over the dataset multiple times
    loss_train = 0
    model.train()

    for sample, target_indices in tqdm(training_iterator, total=len(training_iterator)):
        # -----
        # Inputs
        # -----
        mean_signals  = sample[:,300]
        gaus_expand_to_batch = gaus.expand([len(target_indices), 191, 240, 240])
        mean_signals_expand_to_gaus = mean_signals.expand([240, 240, len(target_indices), 191])
        mean_signals_expand_to_gaus = mean_signals_expand_to_gaus.transpose(0,2).transpose(1,3)
        inputs = module.select_type_inputs(inputtype, gaus_expand_to_batch, mean_signals_expand_to_gaus)
        inputs = inputs.to(cuda0)

        # -----
        # Targets
        # -----
        target_batch = seen_images[target_indices]
        target_batch = target_batch.transpose(3,1).transpose(2,3)
        target_batch = target_batch.to(cuda0)

        # -----
        # Outputs
        # -----
        optimizer.zero_grad()
        y = model(inputs) 

        # -----
        # Before calculating loss, make a mask
        # -----
        y *= confidence_mask.expand_as(y)
        target_batch *= confidence_mask.expand_as(target_batch)

        # ------
        # Loss 
        # ------
        train_loss = lossFunction(y, target_batch)

        # ------
        # Backward & update
        # ------
        train_loss.backward()
        optimizer.step()

        # ------
        # Loss 
        # ------
        loss_train += train_loss.sum().item()
    losses_train.append(loss_train/len(training_iterator.sampler))

    with torch.no_grad():
        loss_test = 0
        model.eval()
        for sample, target_indices in tqdm(testing_iterator, total=len(testing_iterator)):
            # -----
            # Inputs
            # -----
            mean_signals  = sample[:,300]
            gaus_expand_to_batch = gaus.expand([len(target_indices), 191, 240, 240])
            mean_signals_expand_to_gaus = mean_signals.expand([240, 240, len(target_indices), 191])
            mean_signals_expand_to_gaus = mean_signals_expand_to_gaus.transpose(0,2).transpose(1,3)

            inputs = module.select_type_inputs(inputtype, gaus_expand_to_batch, mean_signals_expand_to_gaus)
            inputs = inputs.to(cuda0)
            # -----
            # Targets
            # -----
            target_batch = seen_images[target_indices]
            target_batch = target_batch.transpose(3,1).transpose(2,3)
            target_batch = target_batch.to(cuda0)
            # -----
            # Outputs
            # -----
            y = model(inputs)
            y *= confidence_mask.expand_as(y)
            target_batch *= confidence_mask.expand_as(target_batch)
            # ------
            # Loss 
            # ------
            test_loss = lossFunction(y, target_batch)
            loss_test += test_loss.sum().item()

    losses_test.append(loss_test/len(testing_iterator.sampler))

  0%|          | 0/6162 [00:00<?, ?it/s]

x: torch.Size([4, 2, 240, 240])
conv1_2: torch.Size([4, 64, 120, 120])
conv2_2: torch.Size([4, 128, 60, 60])
conv3: torch.Size([4, 256, 30, 30])
conv4: torch.Size([4, 512, 15, 15])
conv5: torch.Size([4, 512, 7, 7])
resblock: torch.Size([4, 512, 7, 7])
unpool5_want15 torch.Size([4, 512, 15, 15])
deconv5: torch.Size([4, 512, 15, 15])
unpool4 torch.Size([4, 512, 30, 30])
deconv4: torch.Size([4, 256, 30, 30])
unpool3 torch.Size([4, 256, 60, 60])
deconv3: torch.Size([4, 128, 60, 60])
unpool2 torch.Size([4, 128, 120, 120])
deconv2: torch.Size([4, 64, 120, 120])
unpool1 torch.Size([4, 64, 240, 240])
deconv1: torch.Size([4, 3, 240, 240])
final_shape: torch.Size([4, 3, 240, 240])


  0%|          | 2/6162 [00:01<1:11:35,  1.43it/s]

x: torch.Size([4, 2, 240, 240])
conv1_2: torch.Size([4, 64, 120, 120])
conv2_2: torch.Size([4, 128, 60, 60])
conv3: torch.Size([4, 256, 30, 30])
conv4: torch.Size([4, 512, 15, 15])
conv5: torch.Size([4, 512, 7, 7])
resblock: torch.Size([4, 512, 7, 7])
unpool5_want15 torch.Size([4, 512, 15, 15])
deconv5: torch.Size([4, 512, 15, 15])
unpool4 torch.Size([4, 512, 30, 30])
deconv4: torch.Size([4, 256, 30, 30])
unpool3 torch.Size([4, 256, 60, 60])
deconv3: torch.Size([4, 128, 60, 60])
unpool2 torch.Size([4, 128, 120, 120])
deconv2: torch.Size([4, 64, 120, 120])
unpool1 torch.Size([4, 64, 240, 240])
deconv1: torch.Size([4, 3, 240, 240])
final_shape: torch.Size([4, 3, 240, 240])


  0%|          | 3/6162 [00:01<55:57,  1.83it/s]  

...

 14%|█▍        | 893/6162 [03:14<19:11,  4.58it/s]

x: torch.Size([4, 2, 240, 240])
conv1_2: torch.Size([4, 64, 120, 120])
conv2_2: torch.Size([4, 128, 60, 60])
conv3: torch.Size([4, 256, 30, 30])
conv4: torch.Size([4, 512, 15, 15])
conv5: torch.Size([4, 512, 7, 7])
resblock: torch.Size([4, 512, 7, 7])
unpool5_want15 torch.Size([4, 512, 15, 15])
deconv5: torch.Size([4, 512, 15, 15])
unpool4 torch.Size([4, 512, 30, 30])
deconv4: torch.Size([4, 256, 30, 30])
unpool3 torch.Size([4, 256, 60, 60])
deconv3: torch.Size([4, 128, 60, 60])
unpool2 torch.Size([4, 128, 120, 120])
deconv2: torch.Size([4, 64, 120, 120])
unpool1 torch.Size([4, 64, 240, 240])
deconv1: torch.Size([4, 3, 240, 240])
final_shape: torch.Size([4, 3, 240, 240])
x: torch.Size([4, 2, 240, 240])
conv1_2: torch.Size([4, 64, 120, 120])
conv2_2: torch.Size([4, 128, 60, 60])
conv3: torch.Size([4, 256, 30, 30])
conv4: torch.Size([4, 512, 15, 15])
conv5: torch.Size([4, 512, 7, 7])
resblock: torch.Size([4, 512, 7, 7])
unpool5_want15 torch.Size([4, 512, 15, 15])
deconv5: torch.Size([4, 

 15%|█▍        | 894/6162 [03:14<18:27,  4.76it/s]

deconv2: torch.Size([4, 64, 120, 120])
unpool1 torch.Size([4, 64, 240, 240])
deconv1: torch.Size([4, 3, 240, 240])
final_shape: torch.Size([4, 3, 240, 240])
x: torch.Size([4, 2, 240, 240])
conv1_2: torch.Size([4, 64, 120, 120])
conv2_2: torch.Size([4, 128, 60, 60])
conv3: torch.Size([4, 256, 30, 30])
conv4: torch.Size([4, 512, 15, 15])
conv5: torch.Size([4, 512, 7, 7])
resblock: torch.Size([4, 512, 7, 7])
unpool5_want15 torch.Size([4, 512, 15, 15])
deconv5: torch.Size([4, 512, 15, 15])
unpool4 torch.Size([4, 512, 30, 30])
deconv4: torch.Size([4, 256, 30, 30])
unpool3 torch.Size([4, 256, 60, 60])
deconv3: torch.Size([4, 128, 60, 60])
unpool2 torch.Size([4, 128, 120, 120])
deconv2: torch.Size([4, 64, 120, 120])
unpool1 torch.Size([4, 64, 240, 240])
deconv1: torch.Size([4, 3, 240, 240])
final_shape: torch.Size([4, 3, 240, 240])


 15%|█▍        | 895/6162 [03:14<17:58,  4.88it/s]